# ETL Processes
This Notebook describes the step by step etl process used in developing `etl.py`

In [498]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [657]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [500]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
Process for ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [501]:
filepath = 'data/song_data'

In [502]:
song_files = get_files(filepath)

In [503]:
print(len(song_files)) 

71


In [504]:
print(song_files[0])

/Users/shilpamadini/Desktop/DEND/sparkify/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [505]:
df = pd.read_json(song_files[0],lines= True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [506]:
song_data = df[['song_id','title','artist_id','year','duration']].values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [507]:
song_table_insert = ("INSERT INTO songs(song_id, title, artist_id, year, duration) \
                         VALUES(%s,%s,%s,%s,%s) ON CONFLICT(song_id) DO UPDATE \
                         SET title = EXCLUDED.title, artist_id = EXCLUDED.artist_id, \
                         year = EXCLUDED.year, duration = EXCLUDED.duration")

In [514]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [509]:
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [511]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [512]:
artist_table_insert = ("INSERT INTO artists (artist_id, name,location,latitude,longitude) \
                         VALUES(%s,%s,%s,%s,%s) ON CONFLICT(artist_id) DO UPDATE \
                         SET name = EXCLUDED.name, location = EXCLUDED.location, \
                         latitude = EXCLUDED.latitude, longitude = EXCLUDED.longitude")

In [513]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
Process for ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [515]:
filepath = filepath = 'data/log_data'

In [516]:
log_files = get_files(filepath)

In [566]:
log_files

['/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-01-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-02-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-03-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-04-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-05-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-06-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-07-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-08-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-09-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-10-events.json',
 '/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-11-events.json',

In [517]:
print(len(log_files))

30


In [518]:
df = pd.read_json(log_files[0],lines= True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [519]:
df = df.loc[df['page']=='NextSong']
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [539]:
df['ts'] = df['ts'].apply(pd.to_datetime)
t =df['ts']
t.head(2)

2   1970-01-01 00:25:41.106106796
4   1970-01-01 00:25:41.106352796
Name: ts, dtype: datetime64[ns]

In [540]:
timestamp = t.values.tolist()
hour = t.dt.hour.values.tolist()
day = t.dt.day.values.tolist()
week = t.dt.week.values.tolist()
month = t.dt.month.values.tolist()
year = t.dt.year.values.tolist()
weekday = t.dt.weekday.values.tolist()

In [522]:
time_data = list(zip(timestamp,hour,day,week,month,year,weekday))
time_data

[(1541106106796, 0, 1, 1, 1, 1970, 3),
 (1541106352796, 0, 1, 1, 1, 1970, 3),
 (1541106496796, 0, 1, 1, 1, 1970, 3),
 (1541106673796, 0, 1, 1, 1, 1970, 3),
 (1541107053796, 0, 1, 1, 1, 1970, 3),
 (1541107493796, 0, 1, 1, 1, 1970, 3),
 (1541107734796, 0, 1, 1, 1, 1970, 3),
 (1541108520796, 0, 1, 1, 1, 1970, 3),
 (1541109125796, 0, 1, 1, 1, 1970, 3),
 (1541109325796, 0, 1, 1, 1, 1970, 3),
 (1541110994796, 0, 1, 1, 1, 1970, 3)]

In [523]:
column_labels = ['timestamp','hour', 'day','week','month','year','weekday']

In [524]:
time_df = pd.DataFrame(time_data,columns=column_labels)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,1541106106796,0,1,1,1,1970,3
1,1541106352796,0,1,1,1,1970,3
2,1541106496796,0,1,1,1,1970,3
3,1541106673796,0,1,1,1,1970,3
4,1541107053796,0,1,1,1,1970,3


In [525]:
time_df['timestamp']= time_df['timestamp'].apply(pd.to_datetime)

In [526]:
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,1970-01-01 00:25:41.106106796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.106352796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.106496796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.106673796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.107053796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [527]:
time_table_insert = ("INSERT INTO time( start_time, hour, day, week, month, year, weekday) \
                       VALUES(%s,%s,%s,%s,%s,%s,%s) ON CONFLICT DO NOTHING")

In [528]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [529]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [530]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [531]:
user_df.drop_duplicates(keep=False, inplace=True)

/Users/shilpamadini/anaconda/envs/dand_py3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [532]:
user_table_insert = ("INSERT INTO users( user_id, first_name, last_name, gender, level) \
                       VALUES(%s,%s,%s,%s,%s) ON CONFLICT(user_id) DO UPDATE \
                       SET first_name = EXCLUDED.first_name, last_name = EXCLUDED.last_name, \
                       gender = EXCLUDED.gender, level = EXCLUDED.level")

In [533]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [626]:
song_select = ("SELECT SONGS.song_id,ARTISTS.artist_id from SONGS JOIN ARTISTS ON SONGS.artist_id = ARTISTS.artist_id \
                WHERE \
                SONGS.title = %s \
                AND ARTISTS.name = %s \
                AND SONGS.duration = %s\
                ")

In [535]:
df[['song','artist','length']]

,song,artist,length
2,You Gotta Be,Des'ree,246.30812
4,Flat 55,Mr Oizo,144.03873
5,Quem Quiser Encontrar O Amor,Tamba Trio,177.18812
6,Eriatarka,The Mars Volta,380.42077
7,Becoming Insane,Infected Mushroom,440.26730
8,Congratulations,Blue October / Imogen Heap,241.39710
9,Once again,Girl Talk,160.15628
10,Pump It,Black Eyed Peas,214.93506
12,Nobody Puts Baby In The Corner,Fall Out Boy,200.72444
13,Mango Pickle Down River (With The Wilcannia Mob),M.I.A.,233.71710


In [536]:
songplay_table_insert = ("INSERT INTO songplays(start_time, level, user_id, song_id, artist_id, \
                          session_id, location, user_agent) VALUES(%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT DO NOTHING")

In [546]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song,row.artist,row.length))
    songid, artistid = results if results else None, None
    
    # insert songplay record
    # insert songplay record
    songplay_data = [row.ts, row.level, row.userId, songid, artistid,row.sessionId, row.location, row.userAgent ]
    print(songplay_data)
    # cur.execute(songplay_table_insert, songplay_data)
    # con.commit()

[Timestamp('1970-01-01 00:25:41.106106796'), 'free', 8, None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
[Timestamp('1970-01-01 00:25:41.106352796'), 'free', 8, None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
[Timestamp('1970-01-01 00:25:41.106496796'), 'free', 8, None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
[Timestamp('1970-01-01 00:25:41.106673796'), 'free', 8, None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
[Timestamp('1970-01-01 00:25:41.107053796'), 'free', 8, None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windo

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [659]:
conn.close()

# Implement `etl.py`
Below section used to develop and test the functions used in `etl.py`.

In [547]:
df.shape

(11, 18)

In [548]:
def process_song_file(cur, filepath):
   
    # open song file
    df = pd.read_json(filepath,lines= True)
    
    # insert artist record
    artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
    cur.execute(artist_table_insert, artist_data)
    # insert song record
    song_data = df[['song_id','title','artist_id','year','duration']].values[0].tolist()
    cur.execute(song_table_insert, song_data)
    

In [627]:
def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))
        print('{} file is processed.'.format(datafile))



In [550]:
process_data(cur, conn, filepath='data/song_data', func=process_song_file)

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file

In [628]:
songplay_table_insert = ("INSERT INTO songplays(start_time, level, user_id, song_id, artist_id, \
                          session_id, location, user_agent) VALUES(%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT DO NOTHING")

In [655]:
def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath,lines= True)

    # filter by NextSong action
    df = df.loc[df['page']=='NextSong']

    # convert timestamp column to datetime
    df['ts'] = df['ts'].apply(pd.to_datetime)
    t = df['ts']
    
    # retrieve time data records
    timestamp = t.values.tolist()
    hour = t.dt.hour.values.tolist()
    day = t.dt.day.values.tolist()
    week = t.dt.week.values.tolist()
    month = t.dt.month.values.tolist()
    year = t.dt.year.values.tolist()
    weekday = t.dt.weekday.values.tolist()
    
    # insert time data records
    time_data = list(zip(timestamp,hour,day,week,month,year,weekday)) 
    column_labels = ['timestamp','hour', 'day','week','month','year','weekday']
    time_df = pd.DataFrame(time_data,columns=column_labels)
    time_df['timestamp']= time_df['timestamp'].apply(pd.to_datetime)

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[['userId','firstName','lastName','gender','level']] 
    
    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)
    
    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        results = cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        if results:
            songid = results[0]
            artistid = results[1]
        else:
            songid = None
            artistid = None

        # insert songplay record
        songplay_data = [row.ts, row.level, row.userId, songid, artistid,row.sessionId, row.location, row.userAgent ]
        cur.execute(songplay_table_insert, songplay_data)

In [622]:
song_select1 = ("SELECT SONGS.song_id , SONGS.artist_id from SONGS WHERE SONGS.title = 'Setanta matins' \
                ")

In [651]:
def select_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath,lines= True)

    # filter by NextSong action
    df = df.loc[df['page']=='NextSong']

    # convert timestamp column to datetime
    df['ts'] = df['ts'].apply(pd.to_datetime)
    
    for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
        results = cur.execute(song_select1)
        results = cur.fetchone()
        print(results)
        if results:
            songid = results[0]
            artistid = results[1]
        else:
            songid = None
            artistid = None
    
    # insert songplay record
    # insert songplay record
        songplay_data = [row.ts, row.level, row.userId, songid, artistid,row.sessionId, row.location, row.userAgent ]
        print(songplay_data)
    
    

In [652]:
process_data(cur, conn, filepath='data/log_data', func=select_log_file)

30 files found in data/log_data
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.106106796'), 'free', 8, 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.106352796'), 'free', 8, 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.106496796'), 'free', 8, 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.106

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.207150796'), 'free', '53', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 52, 'Klamath Falls, OR', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.207316796'), 'free', '53', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 52, 'Klamath Falls, OR', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.207546796'), 'free', '53', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 52, 'Klamath Falls, OR', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.207775796'), 'free', '53', 'SOZCTXZ12AB0182364', 'SOZCTXZ12

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.375113796'), 'free', '83', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 186, 'Lubbock, TX', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
4/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-04-events.json file is processed.
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.377992796'), 'free', '69', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 256, 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.381242796'), 'paid', '44', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 237, 'Waterloo-Cedar Falls, IA', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:31.0) Gecko/20100101 F

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.431427796'), 'paid', '24', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 23, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.431454796'), 'paid', '44', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 269, 'Waterloo-Cedar Falls, IA', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:31.0) Gecko/20100101 Firefox/31.0']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.431481796'), 'paid', '97', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 147, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.431670796'), 'paid', '24', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB018236

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.519555796'), 'paid', '30', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 130, 'San Jose-Sunnyvale-Santa Clara, CA', 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.519718796'), 'paid', '30', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 130, 'San Jose-Sunnyvale-Santa Clara, CA', 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.519780796'), 'free', '2', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 126, 'Plymouth, IN', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:41.519910796'), 'paid', '30', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 130, 'San Jose-Sunnyvale-Santa Clar

...

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.363215796'), 'paid', '80', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 992, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.363420796'), 'paid', '80', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 992, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.363778796'), 'paid', '80', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 992, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00

[Timestamp('1970-01-01 00:25:43.444959796'), 'paid', '73', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 954, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.445148796'), 'paid', '24', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.445167796'), 'paid', '73', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 954, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.445321796'), 'paid', '24', 'SOZCTXZ12A

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.507055796'), 'paid', '16', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 983, 'Birmingham-Hoover, AL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.507075796'), 'paid', '49', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 1041, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.507201796'), 'paid', '49', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 1041, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.507253796'), 'paid', '16', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 983, 'Birmingham-Hoover, AL', '

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.537327796'), 'free', '91', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 829, 'Dallas-Fort Worth-Arlington, TX', 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.540121796'), 'paid', '73', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.540368796'), 'paid', '73', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.540625796'), 'paid', '73', '

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.599778796'), 'paid', '16', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 1076, 'Birmingham-Hoover, AL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.599840796'), 'paid', '49', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 1114, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.599941796'), 'paid', '49', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 1114, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0']
('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')
[Timestamp('1970-01-01 00:25:43.600033796'), 'paid', '16', 'SOZCTXZ12AB0182364', 'SOZCTXZ12AB0182364', 1076, 'Birmingham-Hoover, AL',

In [658]:
process_data(cur, conn, filepath='data/log_data', func=process_log_file)

30 files found in data/log_data
1/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-01-events.json file is processed.
2/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-02-events.json file is processed.
3/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-03-events.json file is processed.
4/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-04-events.json file is processed.
5/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-05-events.json file is processed.
6/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-06-events.json file is processed.
7/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log_data/2018/11/2018-11-07-events.json file is processed.
8/30 files processed.
/Users/shilpamadini/Desktop/DEND/sparkify/data/log